https://drive.google.com/file/d/1RvLhtV27It_byQwcI70PL8Rxm7q7-VIL/view?usp=sharing

https://pytorch.org/tutorials/recipes/recipes/dynamic_quantization.html

In [ ]:
import torch
from sage.spelling_correction import (
    T5ModelForSpellingCorruption,
    RuM2M100ModelForSpellingCorrection,
    AvailableCorrectors,
)
from datetime import datetime
import os


def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize("temp.p")
    print("model: ", label, " \t", "Size (MB):", size / 1e3 / 1e3)
    os.remove("temp.p")
    return size


corrector = T5ModelForSpellingCorruption.from_pretrained(
    AvailableCorrectors.ent5_large.value
)

corrector.model.eval()
corrector.model.to(torch.device("cpu"))

full_model = print_size_of_model(corrector.model, "fp32")

corrector.model = torch.quantization.quantize_dynamic(
    corrector.model,
    {torch.nn.Linear},
    dtype=torch.qint8,
)

quant_model = print_size_of_model(corrector.model, "int8")
print("{0:.2f} times smaller".format(full_model / quant_model))

model:  fp32  	 Size (MB): 2950.86721

model:  int8  	 Size (MB): 870.149715

3.39 times smaller

In [2]:
start = datetime.now()
metrics = corrector.evaluate(
    "t5_optimization/sage/data/example_data/jfleg",
    batch_size=1,
    prefix="grammar: ",
    # size=3,
)

duration = datetime.now() - start
print(duration)

print(metrics)

100%|██████████| 1602/1602 [24:49<00:00,  1.08it/s]


empty string


100%|██████████| 1602/1602 [00:06<00:00, 247.96it/s]

0:24:55.692000
{'Precision': 82.77, 'Recall': 83.71, 'F1': 83.24}


0:25:23.052333
{'Precision': 82.77, 'Recall': 83.71, 'F1': 83.24}

0:24:55.692000
{'Precision': 82.77, 'Recall': 83.71, 'F1': 83.24}

### default CPU

In [1]:
import torch
from sage.spelling_correction import (
    T5ModelForSpellingCorruption,
    RuM2M100ModelForSpellingCorrection,
    AvailableCorrectors,
)
from datetime import datetime

corrector = T5ModelForSpellingCorruption.from_pretrained(
    AvailableCorrectors.ent5_large.value
)

corrector.model.eval()
corrector.model.to(torch.device("cpu"))

start = datetime.now()
metrics = corrector.evaluate(
    "t5_optimization/sage/data/example_data/jfleg",
    batch_size=1,
    prefix="grammar: ",
    # size=3,
)

duration = datetime.now() - start
print(duration)

print(metrics)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user-name-goes-here/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
100%|██████████| 1602/1602 [1:31:51<00:00,  3.44s/it]


empty string


100%|██████████| 1602/1602 [00:06<00:00, 240.15it/s]

1:31:58.274421
{'Precision': 83.39, 'Recall': 84.25, 'F1': 83.82}


1:31:58.274421
{'Precision': 83.39, 'Recall': 84.25, 'F1': 83.82}